# EOPF SLSTR L1 Product Data Structure Proposal


In [1]:
import os
import xarray as xr
import glob
from utils import get_ds_groups as get_ds
from EOProductDataStructure import EOProductBuilder, EOVariableBuilder, EOGroupBuilder


# Display function

In [2]:
path_to_product = glob.glob("data/S3A_SL_1*.SEN3")[0]
path_to_product
# Groups definition
groups = {}

#groups['coordinates'] = get_ds(path_to_product,["cartesian_"] + ["geodetic_","time_"])
groups['an'] = get_ds(path_to_product, file_pattern="*_an")
groups['bn'] = get_ds(path_to_product, file_pattern="*_bn")
groups['cn'] = get_ds(path_to_product, file_pattern="*_cn")
groups['in'] = get_ds(path_to_product, file_pattern="*_in")

groups['ao'] = get_ds(path_to_product, file_pattern="*_ao")
groups['bo'] = get_ds(path_to_product, file_pattern="*_bo")
groups['co'] = get_ds(path_to_product, file_pattern="*_co")
groups['io'] = get_ds(path_to_product, file_pattern="*_io")
#product.attrs['metadata_files'] = '[xfdumanfist.json]'

IndexError: list index out of range

In [94]:
path_to_product


'data/S3A_SL_1_RBT____20200101T212911_20200101T213211_20200103T011904_0179_053_186_0540_LN2_O_NT_003.SEN3'

# 1. Do nothing : Each netcdf file => group

In [95]:

# Create a new EOProduct instance
product_name = os.path.basename("S3A_SL_1_RBT____")
product = EOProductBuilder("S3A_SL_1_RBT____", EOGroupBuilder('coordinates'))
# do the same work as before 
product.metadata = ["xfdumanifest.xml"]


# global attributes
#get them from a netcdf file 
ds = xr.open_dataset(path_to_product + "/cartesian_an.nc",decode_times=False,mask_and_scale=False)
product.attrs = {att: f'{value[:40]} ...' if isinstance(value, str) else value for att, value in ds.attrs.items()}

for name, subgroup in groups.items():

    # ==================== Product groups setting ========================
    
    for subgroup_name, ds in subgroup.items():
        group = EOGroupBuilder(subgroup_name)
        group.attrs['description'] = f'{subgroup_name} Data Group'
        group.dims = ds.dims

        for v in ds.variables:
            variable = EOVariableBuilder(v, default_attrs = False)
            var = ds.get(v)
            variable.dtype = var.dtype
            variable.dims = var.dims
            variable.attrs = var.attrs
            group.variables.append(variable)

        product.groups.append(group)

product.attrs['metadata_files'] = '[xfdumanfist.xml]'

product.compute()

# 2. Grouping by strips and views

In [96]:

# Create a new EOProduct instance
product_name = os.path.basename("S3A_SL_1_RBT____")
product = EOProductBuilder("S3A_SL_1_RBT____", coords=EOGroupBuilder('coords'))
# do the same work as before 
product.metadatas = ["xfdumanifest.xml"]


# global attributes
#get them from a netcdf file 
ds = xr.open_dataset(path_to_product + "/cartesian_an.nc",decode_times=False,mask_and_scale=False)
product.attrs = {att: f'{value[:40]} ...' if isinstance(value, str) else value for att, value in ds.attrs.items()}

for name, sub_group in groups.items():

    # ==================== Product groups setting ========================
    global_group = EOGroupBuilder(name)
    
    for subgroup_name, ds in sub_group.items():
        group = EOGroupBuilder(subgroup_name)

        group.attrs['description'] = f"{subgroup_name} Data Group"
        group.dims = ds.dims
        global_group.groups.append(group)

        for v in ds.variables:
            variable = EOVariableBuilder(v, default_attrs = False)
            var = ds.get(v)
            variable.dtype = var.dtype
            variable.dims = var.dims
            variable.attrs = var.dims

            group.variables.append(variable)

    product.groups.append(global_group)

product.attrs['metadata_files'] = '[xfdumanfist.xml]'

In [97]:
display(product.compute())

In [98]:

# Create a new EOProduct instance
product_name = os.path.basename("S3A_SL_1_RBT____")
product = EOProductBuilder("S3A_SL_1_RBT____")
# do the same work as before 
product.metadatas = ["xfdumanifest.xml"]

# global attributes
#get them from a netcdf file 
ds = xr.open_dataset(path_to_product + "/cartesian_an.nc",decode_times=False,mask_and_scale=False)
product.attrs = {att: f'{value[:40]} ...' if isinstance(value, str) else value for att, value in ds.attrs.items()}            

for name, sub_group in groups.items():

    # ==================== Product groups setting ========================
    global_group = EOGroupBuilder(name)
    
    coordinates = EOGroupBuilder("coordinates")
    radiances = EOGroupBuilder("radiances")
    bts = EOGroupBuilder("bts")
    qualities = EOGroupBuilder("qualities")
    flags = EOGroupBuilder("flags")
    indices = EOGroupBuilder("indices")
    
    sub_groups = {}
    
    for subgroup_name, ds in groups[name]:
        subgroup_name = subgroup_name.lower()
        
        if "radiance" in subgroup_name:
            group = radiances
            sub_groups["radiances"] = group

        elif "bt" in subgroup_name:
            group = bts
            sub_groups["bts"] = group
        elif "flags" in subgroup_name:
            group = flags
            sub_groups["flags"] = group
        elif "quality" in subgroup_name:
            group = qualities
            sub_groups["qualities"] = group
        elif "indice" in subgroup_name:
            group = indices
            sub_groups["indices"] = group
        else:
            group = coordinates
            global_group.coords = group
        
        #group = EOGroupBuilder(subgroup_name)

        #group.attrs["description"] = f"{subgroup_name} Data Group"
        
        #global_group.groups.append(group)
        
        for d in ds.dims:
            if d not in group.dims:
                group.dims.append(d)


        for v in ds.variables:
            variable = EOVariableBuilder(v,default_attrs = False)
            var = ds.get(v)
            variable.dtype = var.dtype
            variable.dims = var.dims
            variable.attrs = var.attrs
            group.variables.append(variable)

    for gp in sub_groups:
        global_group.groups.append(sub_groups[gp])
        
    product.groups.append(global_group)

product.attrs['metadata_files'] = '[xfdumanfist.xml]'

display(product.compute())

In [99]:

# Create a new EOProduct instance
product_name = os.path.basename("S3A_SL_1_RBT____")
product = EOProductBuilder("S3A_SL_1_RBT____")
# do the same work as before 
product.metadatas = ["xfdumanifest.xml"]


# global attributes
#get them from a netcdf file 
ds = xr.open_dataset(path_to_product + "/cartesian_an.nc",decode_times=False,mask_and_scale=False)
product.attrs = {att: f'{value[:40]} ...' if isinstance(value, str) else value for att, value in ds.attrs.items()}            

for name, sub_group in groups.items():

    # ==================== Product groups setting ========================
    global_group = EOGroupBuilder(name)
    
    for subgroup_name, ds in sub_group.items():
        
        for d in ds.dims:
            if d not in global_group.dims:
                global_group.dims.append(d)

        for v in ds.variables:
            variable = EOVariableBuilder(v, default_attrs = False)
            var = ds.get(v)
            variable.dtype = var.dtype
            variable.dims = var.dims
            variable.attrs = var.attrs

            global_group.variables.append(variable)

    product.groups.append(global_group)

product.attrs['metadata_files'] = '[xfdumanfist.xml]'

In [100]:
display(product.compute())